In [5]:
from __future__ import annotations

from typing import List, Optional
from pydantic import BaseModel, Field, ValidationError

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser


# =============================================================================
# 1) Contratos Pydantic (estructura)
# =============================================================================

class Outline(BaseModel):
    title: str = Field(..., description="Título del tema integrado por las dos ideas")
    audience: str = Field(..., description="Público objetivo: principiante/intermedio/avanzado")
    sections: List[str] = Field(..., description="Secciones principales del outline (5 a 9)")
    key_points: List[str] = Field(..., description="Puntos clave (6 a 12)")
    assumptions: List[str] = Field(..., description="Suposiciones/alcances para no desviarse")


class OutlineReview(BaseModel):
    is_appropriate: bool = Field(..., description="Si el outline es adecuado para el objetivo")
    score: int = Field(..., ge=1, le=10, description="Calificación del outline")
    issues: List[str] = Field(..., description="Problemas detectados (si hay)")
    improvements: List[str] = Field(..., description="Mejoras concretas sugeridas")
    revised_sections: Optional[List[str]] = Field(
        default=None,
        description="Secciones revisadas si NO es apropiado (opcional)",
    )


class FinalArticle(BaseModel):
    title: str
    outline_used: List[str]
    article: str = Field(..., description="Texto final en español, con subtítulos")
    notes: List[str] = Field(..., description="Notas de calidad: coherencia, cobertura, límites")


# =============================================================================
# 2) LLM base
# =============================================================================

llm = ChatOpenAI(model="gpt-4o", temperature=0)


# =============================================================================
# 3) Chain A: Generar Outline estructurado
# =============================================================================

outline_parser = PydanticOutputParser(pydantic_object=Outline)

outline_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Eres un agente consultor experto.\n"
         "Tu tarea: dado 2 ideas relacionadas, proponer un outline de un tema que las integre.\n"
         "Devuelve EXCLUSIVAMENTE la estructura solicitada.\n"
         "{format_instructions}"),
        ("human",
         "Idea 1: {idea1}\n"
         "Idea 2: {idea2}\n"
         "Público: {audience}\n"
         "Objetivo del texto: {goal}\n"
         "Restricciones: {constraints}")
    ]
).partial(format_instructions=outline_parser.get_format_instructions())

outline_chain = outline_prompt | llm | outline_parser


# =============================================================================
# 4) Chain B: Revisar (auto-criticar) el outline y decidir si es apropiado
# =============================================================================

review_parser = PydanticOutputParser(pydantic_object=OutlineReview)

review_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Eres un revisor estricto de outlines.\n"
         "Evalúa si el outline es apropiado para el público/objetivo/restricciones.\n"
         "Si NO es apropiado, propone mejoras concretas y, si es posible, una lista revised_sections.\n"
         "Devuelve EXCLUSIVAMENTE la estructura solicitada.\n"
         "{format_instructions}"),
        ("human",
         "Público: {audience}\n"
         "Objetivo: {goal}\n"
         "Restricciones: {constraints}\n\n"
         "OUTLINE (JSON):\n{outline_json}")
    ]
).partial(format_instructions=review_parser.get_format_instructions())

review_chain = review_prompt | llm | review_parser


# =============================================================================
# 5) Chain C: Escribir el texto final a partir del outline (y revisión)
# =============================================================================

final_parser = PydanticOutputParser(pydantic_object=FinalArticle)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Eres un escritor técnico claro y preciso.\n"
         "Debes escribir un artículo en español siguiendo el outline.\n"
         "Usa subtítulos por sección. Mantén coherencia con el objetivo y restricciones.\n"
         "No inventes datos específicos si no son necesarios; si asumes algo, decláralo.\n"
         "Devuelve EXCLUSIVAMENTE la estructura solicitada.\n"
         "{format_instructions}"),
        ("human",
         "Público: {audience}\n"
         "Objetivo: {goal}\n"
         "Restricciones: {constraints}\n\n"
         "OUTLINE FINAL (JSON):\n{outline_json}\n\n"
         "Si hubo issues en la revisión, tenlos en cuenta:\n{issues}\n"
         "Mejoras sugeridas:\n{improvements}")
    ]
).partial(format_instructions=final_parser.get_format_instructions())

final_chain = final_prompt | llm | final_parser


# =============================================================================
# 6) Orquestación simple (sin loops): outline -> review -> (optional revise) -> final
# =============================================================================

def build_article_from_two_ideas(
    idea1: str,
    idea2: str,
    audience: str = "intermedio",
    goal: str = "Explicar el tema de forma clara y accionable",
    constraints: str = "Extensión: 600-900 palabras. Evita jerga innecesaria. Incluye ejemplos simples.",
) -> FinalArticle:
    """
    No es un agente con bucle; es una mini arquitectura razonadora:
    - Genera un outline tipado
    - Lo evalúa tipado
    - Decide si usar el outline original o uno revisado
    - Genera el texto final tipado
    """
    try:
        outline = outline_chain.invoke(
            {"idea1": idea1, "idea2": idea2, "audience": audience, "goal": goal, "constraints": constraints}
        )
    except ValidationError as e:
        # Fallback seguro si el LLM no respeta el contrato
        raise RuntimeError(f"Error generando outline (contrato no cumplido): {e}")

    review = review_chain.invoke(
        {
            "audience": audience,
            "goal": goal,
            "constraints": constraints,
            "outline_json": outline.model_dump_json(ensure_ascii=False),
        }
    )

    # Decisión (explícita, sin loops): usar outline original o revisado
    final_outline = outline
    if not review.is_appropriate and review.revised_sections:
        final_outline = Outline(
            title=outline.title,
            audience=outline.audience,
            sections=review.revised_sections,
            key_points=outline.key_points,
            assumptions=outline.assumptions,
        )

    article = final_chain.invoke(
        {
            "audience": audience,
            "goal": goal,
            "constraints": constraints,
            "outline_json": final_outline.model_dump_json(ensure_ascii=False),
            "issues": "\n- " + "\n- ".join(review.issues) if review.issues else "Ninguno",
            "improvements": "\n- " + "\n- ".join(review.improvements) if review.improvements else "Ninguna",
        }
    )

    return article


# =============================================================================
# 7) Tests rápidos (múltiples escenarios)
# =============================================================================

def run_tests():
    scenarios = [
        (
            "Agentes con tools", #idea 1
            "Estructura con Pydantic", #idea 2
            "principiante", #audiencia
            "Enseñar cómo un agente usa herramientas y por qué la estructura hace el sistema confiable", #objetivo del texto
        ),
        (
            "Clima promedio vs clima actual", #idea 1
            "Decisión de herramientas (Tavily vs Open-Meteo)", #idea 2
            "intermedio", #audiencia
            "Explicar cómo decidir qué tool usar según la intención del usuario", #objetivo del texto
        ),
        (
            "LangChain chains", #idea 1
            "LangGraph state", #idea 2
            "avanzado", #audiencia
            "Conectar la idea de pipeline lineal con estado y orquestación en grafos", #objetivo del texto
        ),
    ]

    for i, (a, b, audience, goal) in enumerate(scenarios, start=1):
        print("\n" + "=" * 100)
        print(f"SCENARIO {i}: {a} + {b} (audience={audience})")
        article = build_article_from_two_ideas(
            idea1=a,
            idea2=b,
            audience=audience,
            goal=goal,
            constraints="Extensión: 500-700 palabras. Incluye 1 ejemplo concreto. Evita relleno.",
        )
        print("\nTITLE:", article.title)
        print("OUTLINE:", article.outline_used)
        print("\nARTICLE:\n", article.article[:1200], "...\n")  # recorte para consola
        print("NOTES:", article.notes)




In [6]:
run_tests()



SCENARIO 1: Agentes con tools + Estructura con Pydantic (audience=principiante)

TITLE: Integración de Agentes con Tools y Estructuración con Pydantic para Principiantes
OUTLINE: ['Introducción a los Agentes y su Interacción con Herramientas', 'Importancia de la Estructura para la Confiabilidad del Sistema', 'Pydantic: Introducción y Uso Básico', 'Ejemplo Concreto: Implementación de un Agente con Pydantic', 'Beneficios de una Buena Estructura en Sistemas Automatizados', 'Conclusión: Integración de Agentes y Herramientas para Principiantes']

ARTICLE:
 ### Introducción a los Agentes y su Interacción con Herramientas

En el mundo de la programación, un agente es un programa que actúa de manera autónoma para realizar tareas específicas. Estos agentes pueden interactuar con herramientas, que son programas o funciones que les permiten realizar acciones concretas, como acceder a bases de datos o enviar correos electrónicos. La interacción entre agentes y herramientas es fundamental para aut